<a href="https://colab.research.google.com/github/robgon-art/molding-the-imagination/blob/main/Shap_E_Text_to_3d_Dolphin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/openai/shap-e

Cloning into 'shap-e'...
remote: Enumerating objects: 336, done.
remote: Counting objects: 100% (260/260), done.
remote: Compressing objects: 100% (241/241), done.
remote: Total 336 (delta 41), reused 218 (delta 18), pack-reused 76
Receiving objects: 100% (336/336), 11.72 MiB | 16.65 MiB/s, done.
Resolving deltas: 100% (44/44), done.


In [ ]:
%cd shap-e
!pip install -e .

/content/shap-e
Obtaining file:///content/shap-e
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-install-6v8amps2/clip_092c8f3370644aeab2cf122cbe32e9df
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-install-6v8amps2/clip_092c8f3370644aeab2cf122cbe32e9df
  Resolved https://github.com/openai/CLIP.git to commit a1d071733d7111c9c014f024669f959182114e33
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.7/73.7 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 44.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.4/53.4 kB 8.0 MB/s eta 0:00:00
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369497 sha256=720825a4537795d60f51d367ddb68a6cc1bedcb272a4735ca6512afe3d7d3dae


In [ ]:
import sys
import torch
pyt_version_str=torch.__version__.split("+")[0].replace(".", "")
version_str="".join([
    f"py3{sys.version_info.minor}_cu",
    torch.version.cuda.replace(".",""),
    f"_pyt{pyt_version_str}"
])
!pip install fvcore iopath
!pip install --no-index --no-cache-dir pytorch3d -f https://dl.fbaipublicfiles.com/pytorch3d/packaging/wheels/{version_str}/download.html

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fvcore: filename=fvcore-0.1.5.post20221221-py3-none-any.whl size=61400 sha256=80fb7d9e42f7ab5d3b3cb89e7daa7a92842c94ed1c28da14136bf42ce81a46ec
  Stored in directory: /root/.cache/pip/wheels/01/c0/af/77c1cf53a1be9e42a52b48e5af2169d40ec2e89f7362489dd0
  Created wheel for iopath: filename=iopath-0.1.10-py3-none-any.whl size=31532 sha256=1f58ce9d58b3eaf310eea4d09cd5d716c6f76e0990e505c070bec50ce1ea0a09
  Stored in directory: /root/.cache/pip/wheels/9a/a3/b6/ac0fcd1b4ed5cfeb3db92e6a0e476cfd48ed0df92b91080c1d
Successfully built fvcore iopath
Looking in links: https://dl.fbaipublicfiles.com/pytorch3d/packaging/wheels/py310_cu121_pyt210/download.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.3/20.3 MB 290.5 MB/s eta 0:00:00


In [ ]:
from shap_e.util.notebooks import create_pan_cameras, decode_latent_images, gif_widget

In [ ]:
from shap_e.diffusion.gaussian_diffusion import diffusion_from_config
from shap_e.models.download import load_model, load_config
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
xm = load_model('transmitter', device=device)
model = load_model('text300M', device=device)
diffusion = diffusion_from_config(load_config('diffusion'))

  0%|          | 0.00/1.78G [00:00<?, ?iB/s]

100%|████████████████████████████████████████| 890M/890M [00:06<00:00, 150MiB/s]


  0%|          | 0.00/1.26G [00:00<?, ?iB/s]

In [ ]:
import base64
import io
from typing import Union

import ipywidgets as widgets
import numpy as np
import torch
import random
from PIL import Image

from shap_e.models.nn.camera import DifferentiableCameraBatch, DifferentiableProjectiveCamera
from shap_e.models.transmitter.base import Transmitter, VectorDecoder
from shap_e.rendering.torch_mesh import TorchMesh
from shap_e.util.collections import AttrDict

def create_pan_cameras(size: int, num: int, device: torch.device) -> DifferentiableCameraBatch:
    origins = []
    xs = []
    ys = []
    zs = []
    for theta in np.linspace(0, 2 * np.pi, num=num):
        if num == 1:
            theta = 0.15 * np.pi
        z = np.array([np.sin(theta), np.cos(theta), -0.25])
        z /= np.sqrt(np.sum(z**2))
        origin = -z * 4
        x = np.array([np.cos(theta), -np.sin(theta), 0.0])
        y = np.cross(z, x)
        origins.append(origin)
        xs.append(x)
        ys.append(y)
        zs.append(z)
    return DifferentiableCameraBatch(
        shape=(1, len(xs)),
        flat_camera=DifferentiableProjectiveCamera(
            origin=torch.from_numpy(np.stack(origins, axis=0)).float().to(device),
            x=torch.from_numpy(np.stack(xs, axis=0)).float().to(device),
            y=torch.from_numpy(np.stack(ys, axis=0)).float().to(device),
            z=torch.from_numpy(np.stack(zs, axis=0)).float().to(device),
            width=size,
            height=size,
            x_fov=0.7,
            y_fov=0.7,
        ),
    )

In [ ]:
from shap_e.diffusion.sample import sample_latents
import numpy as np
import random
prompt = "a dolphin"
seed = 0
batch_size = 1
guidance_scale = 15
render_mode = 'nerf'
size = 512

torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)

latents = sample_latents(
    batch_size=batch_size, model=model, diffusion=diffusion,
    guidance_scale=guidance_scale, model_kwargs=dict(texts=[prompt] * batch_size),
    progress=True, clip_denoised=True, use_fp16=True, use_karras=True,
    karras_steps = 64, sigma_min=1e-3, sigma_max=160, s_churn=0)

cameras = create_pan_cameras(size, 1, device)
for i, latent in enumerate(latents):
    images = decode_latent_images(xm, latent, cameras, rendering_mode=render_mode)
    display(gif_widget(images))

  0%|          | 0/64 [00:00<?, ?it/s]

HTML(value='<img src="data:image/gif;base64,R0lGODlhAAIAAocAAFCiu1Cfuk6dt02atEuas0masUqYsEqWrUeWrUmTqkSTqkiPpk…

In [ ]:
render_mode = 'nerf' # you can change this to 'stf'
size = 128 #

cameras = create_pan_cameras(size, 15, device)
for i, latent in enumerate(latents):
    images = decode_latent_images(xm, latent, cameras, rendering_mode=render_mode)
    display(gif_widget(images))

HTML(value='<img src="data:image/gif;base64,R0lGODlhgACAAIcAAJ7N5pzM5ZzL45fJ45HE4o3B4Yu/4Ii73IW42oG43IO12X6023…

In [ ]:
# Example of saving the latents as meshes.
from shap_e.util.notebooks import decode_latent_mesh

for i, latent in enumerate(latents):
    t = decode_latent_mesh(xm, latent).tri_mesh()
    with open(f'example_mesh_{i}.ply', 'wb') as f:
        t.write_ply(f)